In [1]:
# Librerias
import random
import numpy as np
from matplotlib import use
import matplotlib.pyplot as plt
import sympy as sp
#from sympy import symbols, Eq, Point3D, Plane, solve, Line3D
import pandas as pd
from shapely.geometry import Polygon, Point

use('Qt5Agg')

In [2]:
inter = None
color_t = None
caras = None
vertex = None
normal = None
M = None
A = None
mp = None
eigenvalues = None
eigenvectors = None
axis = None
traces = None
tri = None
color_m = None
size = None
max_value = 0
x_axis = None
y_axis = None
squares = None

In [3]:
def new_plot():
    global axis
    fig = plt.figure()
    axis = fig.add_subplot(111, projection='3d')

def build_box():
        lines = [
            [0, 1], [0, 2], [0, 4],
            [1, 3], [1, 5], [2, 3],
            [2, 6], [3, 7], [4, 5],
            [4, 6], [5, 7], [6, 7]
        ]
        for line in lines:
            axis.plot(
                [vertex[line[0], 0], vertex[line[1], 0]],
                [vertex[line[0], 1], vertex[line[1], 1]],
                [vertex[line[0], 2], vertex[line[1], 2]],
                color='black'
            )

def print_plot():
    axis.set_xlabel('X')
    axis.set_ylabel('Y')
    axis.set_zlabel('Z')

    axis.legend()
    plt.xlim(x_axis - max_value, x_axis + max_value)
    plt.ylim(y_axis - max_value, y_axis + max_value)
    plt.show()

In [4]:
# Carga los datos de la fractura desde un archivo de texto.
dataset = []
with open("FRAC0003_nrIter4.txt", 'r') as file:
    for line in file:
        values = [float(value) for value in line.split()]
        dataset.append(values)
M = np.array(dataset)

In [5]:
def limits():
    global x_axis, y_axis, max_value    
    min_limits = np.min(M, axis=0)
    max_limits = np.max(M, axis=0)

    for i in range(3):
        max_value = max(max_value, abs(max_limits[i] - min_limits[i]))
    max_value/=2
    x_axis = np.min(M[:, 0]) + max_value/2
    y_axis = np.min(M[:, 1]) + max_value/2
    print(x_axis, y_axis, max_value)
    print(x_axis - max_value, x_axis + max_value)
    print(y_axis - max_value, y_axis + max_value)
limits()

11.54942356075 -2.79233337025 4.8636620595
6.68576150125 16.41308562025
-7.65599542975 2.07132868925


In [6]:
"""
    Calcula la matriz A (matriz de covarianza) de una matriz M y
    los autovalores y autovectores
"""
def load_matrix():
    global A, mp
    print(len(M))
    #M = M[400:440]

    A = np.dot(M.T, M)

    mp = np.mean(M, axis=0)

    eigenvalues, eigenvectors = np.linalg.eig(A)

load_matrix()

80


In [7]:
def get_box():
    """
        Calcula los vértices de la caja que encierra la fractura (matriz m)
    """
    global vertex
    points = M - mp
    _eigenvalues, _eigenvectors = np.linalg.eig(np.cov(points, rowvar=False))
    points = np.dot(points, _eigenvectors)

    # Calcula los límites de la caja en el nuevo sistema de coordenadas
    min_limits = np.min(points, axis=0)
    max_limits = np.max(points, axis=0)

    # Crear puntos de los límites de la caja en el sistema transformado
    vertex = np.array(np.meshgrid([min_limits[0], max_limits[0]], [min_limits[1], max_limits[1]],
                                        [min_limits[2], max_limits[2]])).T.reshape(-1, 3)

    # Transformar los vértices de la caja de vuelta al sistema original
    vertex = np.dot(vertex, _eigenvectors.T) + mp

get_box()

In [8]:
def color_points():
    global color_m, size
    color_m = []
    size = int(len(M) / 20)
    for i in range(size):
        r = [random.random(), random.random(), random.random()]
        color_m.append(r)

    color_m = np.array(color_m)

color_points()

In [9]:
"""
    Representación de los datos en 3D utilizando Matplotlib
"""
def print_points():
    for i in range(size):
        m = M[i * 20:i * 20 + 20]
        axis.scatter(m[:, 0], m[:, 1], m[:, 2], color=color_m[i], s=2)
    

new_plot()
build_box()
print_points()
print_plot()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [10]:
def triangularization():
    """
        Realizar la triangulación de los puntos, teniento en cuenta que cada 20 puntos se apueden realizar
        triangulaiones con los siguientes 2o puntos.

        tri alamacena la dirección de los puntos que forman un triangulo
    """
    global color_t, tri
    color_t = []
    tri = []
    for i in range(0, size - 1):
        for j in range(20):
            if j == 19:
                tri.append([j + (20 * i), j + (20 * (i + 1)), 0 + (20 * (i + 1))])
                tri.append([j + (20 * i), 0 + (20 * (i + 1)), 0 + (20 * i)])
            else:
                tri.append([j + (20 * i), j + (20 * (i + 1)), (j + 1) + (20 * (i + 1))])
                tri.append([j + (20 * i), (j + 1) + (20 * (i + 1)), (j + 1) + (20 * i)])

            color = (color_m[i] + 2 * color_m[i + 1]) / 3
            color_t.append(color)
            color = (2 * color_m[i] + color_m[i + 1]) / 3
            color_t.append(color)
    color_t = np.array(color_t)
    tri = np.array(tri)

triangularization()

In [11]:
"""
    graficar triangularizacion
"""
def print_triangles():
    for s in range(len(tri)):
        triangle = np.array([M[tri[s][0]], M[tri[s][1]], M[tri[s][2]], M[tri[s][0]]])
        axis.plot(triangle[:, 0], triangle[:, 1], triangle[:, 2], color=color_t[s])

new_plot()
build_box()
print_triangles()
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [12]:
def norm():
    """
    sacar la norma de cada triangulo
    """
    global normal
    normal = []
    aux = 0
    for triangle in tri:
        p1, p2, p3 = M[triangle]
        v1 = p2 - p1
        v2 = p3 - p1
        n = np.cross(v1, v2)
        normal.append(n / np.linalg.norm(n))

norm()

In [13]:
def print_norm():
        """
        graficar la norma de los triangulos
        """
        for i in range(len(normal)):
            mp = np.mean(M[tri[i]], axis=0)
            v = 0.3 * normal[i]
            axis.quiver(mp[0], mp[1], mp[2], v[0], v[1], v[2], color='b')

new_plot()
build_box()
print_norm()
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [14]:
def dropnan():
    global tri, normal, color_t    
    df_t = pd.concat([pd.DataFrame(tri), pd.DataFrame(normal), pd.DataFrame(color_t)], axis = 1)
    df_t.columns = ["triangles_x", "triangles_y", "triangles_z",
                    "norm_x", "norm_y", "norm_z",
                    "r", "g", "b"]

    if df_t.isnull().any().any():
        df_t = df_t.dropna()
        tri = df_t[["triangles_x", "triangles_y", "triangles_z"]].to_numpy()
        normal = df_t[["norm_x", "norm_y", "norm_z"]].to_numpy()
        color_t = df_t[["r", "g", "b"]].to_numpy()

dropnan()


In [15]:
caras = []
points = [[0, 1, 2, 3], [0, 1, 4, 5], [0, 4, 2, 6], [7, 5, 3, 1], [7, 6, 3, 2], [7, 6, 5, 4]]
def plane_from_points():
    """
        Calcula los planos a partir de puntos dados y los guarda en la lista.
    """
    # points = [[0, 1, 4, 5]]
    # points = [[0, 1, 2, 3], [0, 1, 4, 5], [0, 4, 2, 6], [7, 5, 3, 1], [7, 6, 3, 2], [7, 6, 5, 4]]
    # points = [[0, 1, 4], [7, 6, 5], [7, 6, 3], [0, 4, 2], [7, 5, 3], [0, 1, 2]]
    for p in points:
        p1, p2, p3 = map(sp.Point3D, (vertex[p[0]], vertex[p[1]], vertex[p[2]]))
        caras.append(sp.Plane(p1, p2, p3))

def line_from_point_and_vector(point, vector):
    """
        Crea una línea a partir de un punto y un vector.

        Args:
        - point: Punto inicial de la línea.
        - vector: Vector director de la línea.

        Returns:
        - Línea creada.
    """
    point = sp.Point3D(point)
    direction_point = point + sp.Point3D(*vector)
    line = sp.Line3D(point, direction_point)
    return line

def direction_points(punto_inicial, vector_director, distancia):
    """
        Calcula un punto final dado un punto inicial, un vector director y una distancia.

        Args:
        - punto_inicial: Punto inicial.
        - vector_director: Vector director.
        - distancia: Distancia desde el punto inicial al punto final.

        Returns:
        - Punto final calculado.
        """
    norma_vector = np.linalg.norm(vector_director)
    direccion = vector_director / norma_vector
    punto_final = punto_inicial + distancia * direccion
    return punto_final

def intersection_plane_line(line):
        """
            Args:
            - line: Línea con la que se calcula la intersección.

            Returns:
            - Lista de puntos de intersección, si hay alguno, o None en caso contrario.
        """
        intersections = []
        for plane in range(len(caras)):
            i = caras[plane].intersection(line)
            if len(i) == 1 and point_in_square(i, points[plane]):  # Solo hay un punto de intersección
                # intersection_point = intersection[0]
                return i
                # return inter
        if len(intersections) > 0:
            return intersections
        else:
            return None
        
def point_in_square(p, s):
    c = [vertex[s[0]], vertex[s[1]], vertex[s[2]], vertex[s[3]]]
    c = [arr.astype(int) for arr in c]
    c = np.array(c)
    point = [np.array([p[0].x]), np.array([p[0].y]), np.array([p[0].z])]
    point = [arr.astype(int) for arr in point]
    point = Point(point)
    polygon = Polygon(c)
    x = polygon.contains(point) or polygon.touches(point)
    return x 

In [16]:
def intersection():
    """
        Calcula las intersecciones entre las normales de los triángulos y la caja, y guarda los puntos en la lista
        inter.
    """
    global inter
    inter = []
    plane_from_points()
    aux = 0
    for i in range(len(normal)):
        mp = np.mean(M[tri[i]], axis=0)
        v = normal[i]
        line = line_from_point_and_vector(mp, v)
        # op = direction_points(mp, v, 1)
        p = intersection_plane_line(line)
        print(aux)
        aux += 1
        if p is None: continue
        p = [p[0].x, p[0].y, p[0].z]
        inter.append(p)
    inter = np.array(inter)

intersection()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [17]:
print(len(inter))

120


In [18]:
def print_intersection():
    # color_t = color_t[:len(inter)]
    axis.scatter(inter[:, 0], inter[:, 1], inter[:, 2], color=color_t, s=2)
new_plot()
build_box()
print_intersection()
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [19]:
def square():
    global squares
    squares = []
    for i in range(0, len(inter), 2):
        if (i + 2) % 40 == 0:
            square = [i, i+1, i+2-40, i+3-40]
        else:
            square = [i, i+1, i+2, i+3]
        squares.append(square)

    squares = np.array(squares)

square()

In [20]:
def print_squares():
    for square in squares:
        s = [inter[square[0]], inter[square[1]], inter[square[3]], inter[square[2]], inter[square[0]]]
        axis.plot([v[0] for v in s], [v[1] for v in s], [v[2] for v in s], color='b')
new_plot()
build_box()
print_squares()
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [42]:
def temp_pp():
    sq = []
    for i in range(41, 120, 2): 
        if (i+1) % 40 == 0:
            print("punto:", i)
            sq.append([i, i-41, i-38, i-79])
        else:
            sq.append([i, i-41, i+2, i-39])
    return np.array(sq)

def print_temp(squares):
    for square in squares:
        s = [inter[square[0]], inter[square[1]], inter[square[3]], inter[square[2]], inter[square[0]]]
        axis.plot([v[0] for v in s], [v[1] for v in s], [v[2] for v in s], color='r')

In [44]:
new_plot()
build_box()
print_points()
print_squares()
sq = temp_pp()
print_temp(sq)
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


punto: 79
punto: 119


In [60]:
mp_s = []
def square_prom():
    global mp_s
    for i in squares:
        mp = np.mean(inter[i], axis=0)
        mp_s.append(mp)
    for i in sq:
        mp = np.mean(inter[i], axis=0)
        mp_s.append(mp)
    mp_s = np.array(mp_s)
   

square_prom()
print(len(mp_s))
print(mp_s[0][0])

100
332042209845807826273277209290493770814622189131587768005149566936175384735029941592489003385771777853215796272417753570642148270474053239377677184044623018209242405015054195671607739230041286631/36322525273339473134058195678375069220195241185224248318326562096877002720972494618433838721750109311141322928026378391291080603961919369673856530443963524150251049378467493495110400000000000000


In [65]:
def print_p_square():
    axis.scatter(mp_s[:, 0], mp_s[:, 1], mp_s[:, 2],s=1, color="brown")

new_plot()
build_box()
print_p_square()
print_squares()
sq = temp_pp()
print_temp(sq)
print_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


punto: 79
punto: 119
